In [1]:
%matplotlib inline
import pyproj
import logging
import argparse
import os
import json
import pandas as pd
import geopandas
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy import crs as ccrs
from shapely import geometry
from asmclib import geohotspot
from datetime import datetime, timedelta
from shapely.geometry import box

In [10]:
hs_grid = geopandas.read_file("./output/NRT_2km_hotspot_grid_20191212.geojson")
hs_grid_fires = hs_grid[hs_grid['adj_weight'] >= 0.31]

In [11]:
hs_grid_fires_84 = hs_grid.to_crs({'init': 'epsg:4326'})

In [12]:
x = hs_grid_fires_84.iloc[0].geometry
x, y = x.exterior.coords.xy
print (x, y)

array('d', [118.02045427784023, 118.03842058352262, 118.03842058352262, 118.02045427784023, 118.02045427784023]) array('d', [28.599776675798687, 28.599776675798687, 28.58400134802416, 28.58400134802416, 28.599776675798687])


In [13]:
f = open('test.ann', "w+")
for index, row in hs_grid_fires_84[hs_grid_fires_84['adj_weight'] >= 0.31].iterrows():
    x, y = row.geometry.exterior.coords.xy
    list_len = len(x)
    f.write('######################################\n')
    f.write('POLYLINE\n')
    f.write('COLOR 255   0   0\n')
    f.write('THICKNESS       2\n')
    f.write('LINESTYLE       1\n')
    f.write('STARTPOINTS\n')
    for i in range(0, list_len):
        f.write('       %f       %f\n' % (x[i], y[i]))
    f.write('ENDPOINTS\n')
for index, row in hs_grid_fires_84[hs_grid_fires_84['adj_weight'] < 0.31].iterrows():
    x, y = row.geometry.exterior.coords.xy
    list_len = len(x)
    f.write('######################################\n')
    f.write('POLYLINE\n')
    f.write('COLOR 255   255   0\n')
    f.write('THICKNESS       2\n')
    f.write('LINESTYLE       1\n')
    f.write('STARTPOINTS\n')
    for i in range(0, list_len):
        f.write('       %f       %f\n' % (x[i], y[i]))
    f.write('ENDPOINTS\n')

f.write('######################################')

38

In [234]:
map_shape_feature = ShapelyFeature(Reader('./shapefile/GADM_Simplified_ASMC_ASEAN_v2.2c.shp').geometries(), ccrs.PlateCarree())

In [261]:
map_left = 90
map_right = 140
map_top = 30
map_bottom = -15

In [ ]:
fig = plt.figure(figsize=(30, 30), dpi=1200, frameon=False)
ax = plt.axes([0, 0, 1, 1], projection=ccrs.Mercator())
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.set_frame_on(False)
ax.set_extent([map_left, map_right, map_bottom, map_top], crs=ccrs.PlateCarree())
ax.add_feature(map_shape_feature, linewidth=0.1, edgecolor='black', facecolor='None', zorder=1)
ax.add_geometries(hs_grid_fires.geometry, alpha=0.8, crs=ccrs.Mercator(), facecolor='none', edgecolor='red', linewidth=0.1, zorder=2)